What features and parameters dictate electric car demand?

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
import psycopg2
import sqlalchemy

In [3]:
# # Loading Data

connection = psycopg2.connect(
    host = 'teamev.c5ap8dqhexya.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'dogecoin',
    database='postgres'
    )
cursor=connection.cursor()

sql = """
SELECT * FROM monthly_sales; 
"""

df = pd.read_sql(sql, con=connection)
df.head(10)

# Split the Data into Training and Testing

In [4]:
# # Create our features
# X = df.drop('<dependent_feature>', axis=1)

# # Create our target
# y = df['<dependent_feature>']

# X = pd.get_dummies(X)

In [5]:
# X.describe()

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(X,
#                                                     y,
#                                                     random_state=1)

In [7]:
# # Resample the training data with the BalancedRandomForestClassifier
# from imblearn.ensemble import BalancedRandomForestClassifier
# brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
# brf_model = brf_model.fit(X_train, y_train)
# brf_model

In [8]:
# # Calculated the balanced accuracy score
# y_pred = brf_model.predict(X_test)
# acc_score = balanced_accuracy_score(y_test, y_pred)
# acc_score

In [ ]:
# # List the features sorted in descending order by feature importance
# importances = rf_model.feature_importances_
# sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)